<a href="https://colab.research.google.com/github/lalitjoshi007/recommend/blob/main/Creato_Recommendation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing necessary libraries

Cloning into 'Creato_Recommendation'...
fatal: could not read Username for 'https://github.com': No such device or address


In [1]:
!pip show tensorflow

Name: tensorflow
Version: 2.17.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras


In [2]:

!pip install tensorflow-recommenders


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.1 MB/s eta 0:00:00


In [3]:
!pip install tensorflow-recommenders scikit-learn scipy

In [45]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cluster import KMeans
from scipy.stats import beta
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import openpyxl
from sklearn.preprocessing import LabelEncoder

Data pre processing

In [31]:
Dummy_data=pd.read_csv("/content/Tables and Attributes - Dummy_data.csv")

In [32]:
Dummy_data.head()

,user_id,name,age,gender,location,preferred_content_type,interests,fav_genres,interaction_rate,time_spent_daily,...,share_count,active_hours,language_preference,device_type,engagement_score,user_role,comment_sentiment,content_visibility,frequent_captions,frequent_hashtags
0,1,Alex Johnson,25,Male,New York,Videos,"Travel, Music","Fashion, Tech",Daily,120,...,5,Evening,English,Mobile,85,Consumer,Positive,Public,"""Living the dream"", ""NY vibes""","#Travel, #MusicLife"
1,2,Priya Singh,30,Female,Mumbai,Images,"Food, Art","Food, Photography",Weekly,90,...,3,Afternoon,"English, Hindi",Mobile,70,Consumer,Neutral,Public,"""Foodie at heart"", ""Yummy!""","#Foodie, #HealthyRecipes"
2,3,Michael Lee,22,Male,London,Reels,"Fitness, Travel","Tech, Adventure",Daily,150,...,10,Morning,English,Desktop,95,Creator,Positive,Public,"""No pain, no gain"", ""Adventure awaits""","#FitnessGoals, #Explore"
3,4,Sara Tanaka,29,Female,Tokyo,Stories,"Fashion, DIY","Fashion, Beauty",Monthly,80,...,2,Evening,"Japanese, English",Mobile,60,Brand,Neutral,Private,"""Handmade with love"", ""Tokyo life""","#DIY, #Handmade"
4,5,Ahmed Karim,35,Male,Dubai,Reels,"Business, Tech","Tech, Business",Daily,110,...,15,Night,"Arabic, English",Mobile,88,Brand,Positive,Public,"""Hustle mode on"", ""Future tech""","#Entrepreneur, #TechTrends"


In [33]:
Dummy_data.shape

(20, 22)

In [34]:
important_Feature=["user_id","name","gender","location","preferred_content_type","interests","fav_genres","interaction_rate","time_spent_daily","content_liked","language_preference","frequent_captions","frequent_hashtags"]
print(important_Feature)

['user_id', 'name', 'gender', 'location', 'preferred_content_type', 'interests', 'fav_genres', 'interaction_rate', 'time_spent_daily', 'content_liked', 'language_preference', 'frequent_captions', 'frequent_hashtags']


In [35]:
#replaceing null values
for feature in important_Feature:
  Dummy_data[feature]=Dummy_data[feature].fillna("")

In [36]:
combined_features = (
    Dummy_data["user_id"].astype(str) +[" "]+
    Dummy_data["location"].astype(str) +[" "]+
    Dummy_data["preferred_content_type"].astype(str) +[" "]+
    Dummy_data["interests"].astype(str) +[" "]+
    Dummy_data["fav_genres"].astype(str) +[" "]+
    Dummy_data["language_preference"].astype(str) +[" "]+
    Dummy_data["interests"].astype(str) +[" "]+
    Dummy_data["language_preference"].astype(str)+[" "]+
    Dummy_data["frequent_captions"].astype(str)+[" "]+
    Dummy_data["frequent_hashtags"].astype(str)
)

In [37]:
print(combined_features)

0     1 New York Videos Travel, Music Fashion, Tech ...
1     2 Mumbai Images Food, Art Food, Photography En...
2     3 London Reels Fitness, Travel Tech, Adventure...
3     4 Tokyo Stories Fashion, DIY Fashion, Beauty J...
4     5 Dubai Reels Business, Tech Tech, Business Ar...
5     6 Beijing Videos Beauty, Travel Travel, Beauty...
6     7 Mexico City Images Photography Adventure, Fo...
7     8 Islamabad Stories Fitness, Fashion Fashion, ...
8     9 Kyoto Videos Gaming, Travel Gaming, Adventur...
9     10 Berlin Reels DIY, Cooking Cooking, Fashion ...
10    11 New Delhi Stories Tech, Sports Tech, Fitnes...
11    12 Los Angeles Videos Music, Dance Music, Life...
12    13 Cairo Images Art, Photography Photography, ...
13    14 Barcelona Reels Travel, Music Adventure, Fo...
14    15 Mumbai Stories Fitness, Tech Fitness, Busin...
15    16 Rome Images Cooking, Fashion Cooking, Lifes...
16    17 Bengaluru Videos Tech, Finance Tech, Busine...
17    18 Paris Stories Fashion, Travel Fashion, 

In [38]:
vectorizer=TfidfVectorizer()

In [39]:
features_vectors=vectorizer.fit_transform(combined_features)

In [40]:
print(features_vectors)

  (0, 115)	0.22970263123486656
  (0, 159)	0.2613181244080856
  (0, 154)	0.16363593601233048
  (0, 145)	0.4909078080369914
  (0, 110)	0.4145421636936456
  (0, 55)	0.16363593601233048
  (0, 138)	0.16363593601233048
  (0, 50)	0.19835399344859436
  (0, 103)	0.22970263123486656
  (0, 141)	0.22970263123486656
  (0, 45)	0.2613181244080856
  (0, 118)	0.2613181244080856
  (0, 153)	0.1898718204007793
  (0, 111)	0.2613181244080856
  (1, 50)	0.167365240483938
  (1, 109)	0.19381629503910147
  (1, 86)	0.16020823342150212
  (1, 62)	0.5246676488162026
  (1, 16)	0.32041646684300423
  (1, 125)	0.17488921627206755
  (1, 82)	0.3497784325441351
  (1, 63)	0.4409850285742336
  (1, 18)	0.19381629503910147
  (1, 81)	0.2204925142871168
  (1, 161)	0.2204925142871168
  :	:
  (18, 127)	0.14843994794561408
  (18, 29)	0.4813606560905176
  (18, 113)	0.1941130759399404
  (18, 131)	0.5823392278198212
  (18, 9)	0.22083014310725893
  (18, 96)	0.22083014310725893
  (18, 137)	0.22083014310725893
  (18, 41)	0.22083014310725

In [50]:
# Create user profile features (e.g., location, language, interaction rate)
user_features = Dummy_data[["location", "language_preference", "interaction_rate", "time_spent_daily"]]
combined_user_features = (
    Dummy_data["location"].astype(str) + " " +
    Dummy_data["language_preference"].astype(str)
)

# TF-IDF Vectorization for user profile
user_vectorizer = TfidfVectorizer(stop_words="english")
user_feature_vectors = user_vectorizer.fit_transform(combined_user_features)


In [41]:
Dummy_data['user_id'] = Dummy_data['user_id'].astype(str)
Dummy_data['reel_id'] = Dummy_data['user_id'] + "_reel"


Content based Recommendation

In [42]:
#  Define the important features for content filtering
content_features = [
    "preferred_content_type", "fav_genres", "language_preference",
    "frequent_captions", "frequent_hashtags"
]

# Replace null values for content-related features as well
for feature in content_features:
    Dummy_data[feature] = Dummy_data[feature].fillna("")

#  Create a combined content profile for each reel based on content features
combined_content_features = (
    Dummy_data["preferred_content_type"].astype(str) + " " +
    Dummy_data["fav_genres"].astype(str) + " " +
    Dummy_data["language_preference"].astype(str) + " " +
    Dummy_data["frequent_captions"].astype(str) + " " +
    Dummy_data["frequent_hashtags"].astype(str)
)

#  Create the content-based feature vectors
content_vectorizer = TfidfVectorizer()
content_feature_vectors = content_vectorizer.fit_transform(combined_content_features)

# Calculate content-based similarity matrix (content filtering)
content_similarity_matrix = cosine_similarity(content_feature_vectors)








Collaborative Filtering using cosine similarity


In [43]:
# Calculate cosine similarity
similarity_matrix = cosine_similarity(features_vectors)

def recommend_reels(user_id, n_recommendations=5, content_weight=0.5):
    user_index = Dummy_data[Dummy_data['user_id'] == str(user_id)].index[0]        # Find the index of the user in the similarity matrix
    similar_users = similarity_matrix[user_index]          # Get the similarity scores of other users to this user
    similar_users = sorted(enumerate(similar_users), key=lambda x: x[1], reverse=True)[1:]        # Sort users based on similarity scores
    top_n_similar_users = similar_users[:n_recommendations]         # Get the top N similar users
    recommended_reels_collab = []         # Get the reels liked by these similar users
    for user_index, similarity_score in top_n_similar_users:
        user_reels = Dummy_data[Dummy_data['user_id'] == Dummy_data.iloc[user_index]['user_id']]['reel_id'].tolist()
        recommended_reels_collab.extend(user_reels)
    recommended_reels_collab = list(set(recommended_reels_collab))[:5] # Remove duplicates and return the top N recommendations
    user_reels = Dummy_data[Dummy_data['user_id']==str(user_id)]['reel_id'].tolist()
    similar_content_scores = []
    for reel_id in user_reels:
      reel_index = Dummy_data[Dummy_data['reel_id'] == reel_id].index[0]
      content_similarities = content_similarity_matrix[reel_index]
      similar_content_scores.extend(list(enumerate(content_similarities)))

    similar_content_scores = sorted(similar_content_scores, key=lambda x: x[1], reverse=True)
    top_content_recommendations = [Dummy_data.iloc[reel_index]['reel_id'] for reel_index, _ in similar_content_scores[:n_recommendations]]


    combined_recommendations = list(set(recommended_reels_collab +top_content_recommendations))[:n_recommendations]


    return combined_recommendations


In [48]:

# Create LabelEncoders for the relevant columns
sentiment_encoder = LabelEncoder()
rate_encoder = LabelEncoder()

Dummy_data['comment_sentiment_encoded'] = sentiment_encoder.fit_transform(Dummy_data['comment_sentiment'])
Dummy_data['interaction_rate_encoded'] = rate_encoder.fit_transform(Dummy_data['interaction_rate'])

# Update interaction_data and target to use the encoded columns
interaction_data = Dummy_data[[ "share_count", "time_spent_daily", "comment_sentiment_encoded"]]
target = Dummy_data["interaction_rate_encoded"]

# Fit the GBDT model
gbdt_model = GradientBoostingClassifier()
gbdt_model.fit(interaction_data, target)

# Predict interaction probability for each video
interaction_predictions = gbdt_model.predict_proba(interaction_data)[:, 1]


In [56]:
def rank_reels_with_gbdt(reels, interaction_predictions):
    """Ranks reels based on predicted interaction probabilities using the GBDT model."""
    reel_interaction_scores = {reel_id: interaction_predictions[Dummy_data[Dummy_data['reel_id'] == reel_id].index[0]] for reel_id in reels}
    ranked_reels = sorted(reel_interaction_scores, key=reel_interaction_scores.get, reverse=True)
    return ranked_reels

In [51]:
from sklearn.cluster import KMeans

# Cluster users based on their interaction features
kmeans = KMeans(n_clusters=5)  # Number of clusters can be adjusted
user_clusters = kmeans.fit_predict(user_feature_vectors)

# Assign users to clusters and refine recommendations based on their cluster
Dummy_data['user_cluster'] = user_clusters


In [52]:
import random

def diversify_recommendations(recommended_reels):
    # Introduce randomness to explore different content
    if random.random() < 0.2:  # 20% chance to introduce diversity
        # Pick some random reels that are not in the user's history
        new_reels = Dummy_data[~Dummy_data['reel_id'].isin(recommended_reels)]['reel_id'].sample(3).tolist()
        recommended_reels.extend(new_reels)
    return list(set(recommended_reels))[:5]


In [53]:
def recommend_reels(user_id, n_recommendations=5, content_weight=0.5):
    # Collaborative filtering part
    user_index = Dummy_data[Dummy_data['user_id'] == str(user_id)].index[0]
    similar_users = similarity_matrix[user_index]
    similar_users = sorted(enumerate(similar_users), key=lambda x: x[1], reverse=True)[1:]
    top_n_similar_users = similar_users[:n_recommendations]

    recommended_reels_collab = []
    for user_index, similarity_score in top_n_similar_users:
        user_reels = Dummy_data[Dummy_data['user_id'] == Dummy_data.iloc[user_index]['user_id']]['reel_id'].tolist()
        recommended_reels_collab.extend(user_reels)

    # Content-based filtering part
    user_reels = Dummy_data[Dummy_data['user_id'] == str(user_id)]['reel_id'].tolist()
    similar_content_scores = []
    for reel_id in user_reels:
        reel_index = Dummy_data[Dummy_data['reel_id'] == reel_id].index[0]
        content_similarities = content_similarity_matrix[reel_index]
        similar_content_scores.extend(list(enumerate(content_similarities)))

    similar_content_scores = sorted(similar_content_scores, key=lambda x: x[1], reverse=True)
    top_content_recommendations = [Dummy_data.iloc[reel_index]['reel_id'] for reel_index, _ in similar_content_scores[:n_recommendations]]

    # Combine both recommendations and rank them
    combined_recommendations = list(set(recommended_reels_collab + top_content_recommendations))[:n_recommendations]

    # Rank using GBDT model
    ranked_reels = rank_reels_with_gbdt(combined_recommendations, interaction_predictions)

    # Introduce exploration
    diversified_reels = diversify_recommendations(ranked_reels)

    return diversified_reels


In [59]:

user_id = int(input("Enter the user id: "))
recommend = recommend_reels(user_id)
print("The recommended reels for user ID {} are:".format(user_id))
print(recommend)


for reel_id in recommend:
    reel_info = Dummy_data[Dummy_data['reel_id'] == reel_id]
    print(reel_info[['reel_id', 'frequent_captions', 'frequent_hashtags', 'time_spent_daily']])


Enter the user id: 1
The recommended reels for user ID 1 are:
['9_reel', '1_reel', '12_reel', '17_reel', '18_reel']
  reel_id           frequent_captions     frequent_hashtags  time_spent_daily
8  9_reel  "Game on!", "Travel vibes"  #Gamer, #TravelJapan                70
  reel_id               frequent_captions    frequent_hashtags  \
0  1_reel  "Living the dream", "NY vibes"  #Travel, #MusicLife   

   time_spent_daily  
0               120  
    reel_id                  frequent_captions     frequent_hashtags  \
11  12_reel  "Dance till you drop", "LA vibes"  #DanceLife, #LAStyle   

    time_spent_daily  
11               125  
    reel_id                      frequent_captions    frequent_hashtags  \
16  17_reel  "Living my best life", "Dreaming big"  #Inspo, #LifeIsGood   

    time_spent_daily  
16               115  
    reel_id             frequent_captions           frequent_hashtags  \
17  18_reel  "Weekend vibes", "City life"  #UrbanAdventure, #CityLove   

    time_spent_d